<a href="https://colab.research.google.com/github/akib1162100/AMT_R-D/blob/r%26d/AMT_chord_variation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/MyDrive')

audio_file = "/content/Bruno Mars - When I was Your Man (Acapella - Vocals Only).mp3"


In [ ]:
#need to set the BPM for the song
#this will be selected according to the genre of the song

#Note: If you want librosa to suggest a bpm for the song then state None or 0

estimated_tempo = 73

# Audio to Midi

In [ ]:
#Import the required libraries
!pip install midiutil
!pip install pretty_midi
import librosa
from librosa import display
import numpy as np
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import mido
import pretty_midi
from mido import MidiFile, MidiTrack, Message, MetaMessage
from midiutil.MidiFile import MIDIFile
import time
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54570 sha256=d320bb53cebc6d519e3ec750c36af7c56cae10917e50bdbcecf0f6999101197a
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=d3678e8d2417235ea9c1b631f139af6de7d393c15ec1844f270c6fd14ed72a8b
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [ ]:
#load the audio, and detect the pitch
y, sr = librosa.load(audio_file, sr = None) #The raw audio data signal is stored in y
duration = librosa.get_duration(y = y, sr = sr)
x = 10
frame_size = 2**x
z = 1/2
hop_length = int(z*frame_size)

y_seg_clean = librosa.effects.split(y = y, frame_length = frame_size, hop_length = hop_length) #Split an audio signal into non-silent intervals.
y_clean = librosa.effects.remix(y = y, intervals= y_seg_clean) #Remix an audio signal by re-ordering time intervals.
harmonic, percussive = librosa.effects.hpss(y_clean) #Decompose an audio time series into harmonic and percussive components.
onset_env = librosa.onset.onset_strength(y = harmonic, sr = sr, hop_length=hop_length) #Compute a spectral flux onset strength envelope.

onset_samples = librosa.onset.onset_detect(y=harmonic,
                                           onset_envelope = onset_env,
                                           sr=sr, units='samples',
                                           hop_length=hop_length,
                                           backtrack=True,
                                           pre_max=20,
                                           post_max=20,
                                           pre_avg=80,
                                           post_avg=80,
                                           delta = 0.001,
                                           wait=0)
onset_boundaries = np.concatenate([onset_samples, [len(harmonic)]])
onset_times = librosa.samples_to_time(onset_boundaries, sr=sr)

#librosa decides the tempo if provided value is None
tempo = int(librosa.beat.tempo(onset_envelope = onset_env, sr = sr )) if estimated_tempo == 0 or estimated_tempo == None else int(estimated_tempo)

def estimate_pitch(segment, sr, fmin=50.0, fmax=2000.0):

    r = librosa.autocorrelate(segment)#stft

    i_min = sr/fmax
    i_max = sr/fmin
    r[:int(i_min)] = 0
    r[int(i_max):] = 0
    i = r.argmax()
    f0 = float(sr)/i
    return f0

def generate_sine(f0, sr, n_duration):
    n = np.arange(n_duration)
    return 0.2*np.sin(2*np.pi*f0*n/float(sr))


def estimate_vol(segment, sr, fmin=50.0, fmax=2000.0):
    vol = librosa.feature.rms(y = segment)
    vol_avg = np.mean(vol)
    return vol_avg

def estimate_pitch_and_generate_sine(x, onset_samples, i, sr):
    n0 = onset_samples[i]
    n1 = onset_samples[i+1]
    f0 = estimate_pitch(x[n0:n1], sr)#segment of the frequencies
    vol = estimate_vol(x[n0:n1], sr)
    return generate_sine(f0, sr, n1-n0) , librosa.hz_to_note(f0), vol, f0

y_pure = np.concatenate([
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[0]
    for i in range(len(onset_boundaries)-1)
])
y_notes = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[1]
    for i in range(len(onset_boundaries)-1)
]
y_vol = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[2]
    for i in range(len(onset_boundaries)-1)
]
freq = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[3]
    for i in range(len(onset_boundaries)-1)
]



degrees = [librosa.note_to_midi(note) if note!='0' else 0 for i,note in enumerate(y_notes)]
beats_per_sec = tempo/60
start_times_beat = [onset*beats_per_sec for onset in onset_times]
duration_in_beat = [start_times_beat[i]-start_times_beat[i-1] for i in range(1,len(start_times_beat))]
start_times_beat.pop()
norm_vol = [100] * (len(degrees))
# norm_vol = [int(vol/max(y_vol)*127) for vol in y_vol]

In [ ]:
#1. identify notes from the pitch
notes_by_pitch = {}
for note, duration in zip(degrees,duration_in_beat):
    if note in notes_by_pitch: #finds the unique notes
        notes_by_pitch[note].append(duration)
    else:
        notes_by_pitch[note] = [duration]

unique_notes = list(librosa.midi_to_note(list(notes_by_pitch.keys())))

#identify occurences of each of these notes
note_occurences = []
for key in notes_by_pitch:
  occurences = len(notes_by_pitch[key])
  note_occurences.append(occurences)

#sort them according to note_occurences
note_data = list(zip(unique_notes, note_occurences))
note_desc = sorted(note_data, key = lambda x: x[1], reverse = True)

#Find the note that occurs the most, this will later be used to determine the octave of the chords
most_common_note = max(notes_by_pitch, key = lambda k: len(notes_by_pitch[k]))
key = librosa.midi_to_note(most_common_note)

In [ ]:
#Dictionary of all possible keys that a song can be in
key_scales = {

    'C Major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D Major': ['D', 'E', 'F♯', 'G', 'A', 'B', 'C♯'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'E Major': ['E', 'F♯', 'G♯', 'A', 'B', 'C♯', 'D♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'F Major': ['F', 'G', 'A', 'Bb', 'C', 'D', 'E'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G Major': ['G', 'A', 'B', 'C', 'D', 'E', 'F♯'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A Major': ['A', 'B', 'C♯', 'D', 'E', 'F♯', 'G♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'B Major': ['B', 'C♯', 'D♯', 'E', 'F♯', 'G♯', 'A♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/

    'C♯ Major': ['C♯', 'D♯', 'E♯', 'F♯', 'G♯', 'A♯', 'B♯'],# https://www.piano-keyboard-guide.com/key-of-c-sharp.html
    'D♯ Major': ['D♯', 'E♯', 'F♯♯', 'G♯', 'A♯', 'B♯', 'C♯♯'],# https://themusicambition.com/d-sharp-major-scale/
    'F♯ Major': ['F♯', 'G♯', 'A♯', 'B', 'C♯', 'D♯', 'E♯'],# https://www.piano-keyboard-guide.com/f-sharp-major-scale.html
    'G♯ Major': ['G♯', 'A♯', 'B♯', 'C♯', 'D♯', 'E♯', 'F♯♯'],# https://themusicambition.com/g-sharp-major-scale/
    'A♯ Major': ['A♯', 'B♯', 'C♯♯', 'D♯', 'E♯', 'F♯♯', 'G♯♯'],# https://themusicambition.com/a-sharp-major-scale/

    'C Minor': ['C', 'D', 'Eb', 'F', 'G', 'Ab', 'Bb'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D Minor': ['D', 'E', 'F', 'G', 'A', 'Bb', 'C'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'E Minor': ['E', 'F♯', 'G', 'A', 'B', 'C', 'D'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'F Minor': ['F', 'G', 'Ab', 'Bb', 'C', 'Db', 'Eb'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G Minor': ['G', 'A', 'Bb', 'C', 'D', 'Eb', 'F'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A Minor': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],#  https://vitapiano.com/the-complete-guide-to-piano-scales/
    'B Minor': ['B', 'C♯', 'D', 'E', 'F♯', 'G', 'A'],# https://vitapiano.com/the-complete-guide-to-piano-scales/

    'C♯ Minor': ['C♯', 'D♯', 'E', 'F♯', 'G♯', 'A', 'B'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D♯ Minor': ['D♯', 'E♯', 'F♯', 'G♯', 'A♯', 'B', 'C♯'],# https://www.piano-keyboard-guide.com/key-of-d-sharp-minor.html
    'F♯ Minor': ['F♯', 'G♯', 'A', 'B', 'C♯', 'D', 'E'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G♯ Minor': ['G♯', 'A♯', 'B', 'C♯', 'D♯', 'E', 'F♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A♯ Minor': ['A♯', 'B♯', 'C♯', 'D♯', 'E♯', 'F♯', 'G♯'],# https://www.piano-keyboard-guide.com/key-of-a-sharp-minor.html

    'Cb Major': ['Cb', 'Db', 'Eb', 'Fb', 'Gb', 'Ab', 'Bb'], #https://www.basicmusictheory.com/c-flat-major-triad-chords
    'Db Major': ['Db', 'Eb', 'F', 'Gb', 'Ab', 'Bb', 'C'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Eb Major': ['Eb', 'F', 'G', 'Ab', 'Bb', 'C', 'D'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Gb Major': ['Gb', 'Ab', 'Bb', 'Cb', 'Db', 'Eb', 'F'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Ab Major': ['Ab', 'Bb', 'C', 'Db', 'Eb', 'F', 'G'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Bb Major': ['Bb', 'C', 'D', 'Eb', 'F', 'G', 'A'],# https://vitapiano.com/the-complete-guide-to-piano-scales/


    'Bb Minor': ['Bb', 'C', 'Db', 'Eb', 'F', 'Gb', 'Ab'],#  https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Eb Minor': ['Eb', 'F', 'Gb', 'Ab', 'Bb', 'Cb', 'Db'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Ab Minor': ['Ab', 'Bb', 'Cb', 'Db', 'Eb', 'Fb', 'Gb'],# https://www.basicmusictheory.com/a-flat-minor-triad-chords

}


In [ ]:
# Identify the relative keys (keys with the same musical notes but in different order)
value_to_keys = {}
for key, value in key_scales.items():
    value_key = frozenset(value)  # Convert the list of notes to a frozenset
    if value_key in value_to_keys:
        value_to_keys[value_key].append(key)
    else:
        value_to_keys[value_key] = [key]

# Extract the keys with the same values
keys_with_same_values = [keys for keys in value_to_keys.values() if len(keys) > 1] #greater than 1, i.e more than one key for the same combination of notes

# Print the keys that have the same values
for keys in keys_with_same_values:
    print("The relative keys:", keys)

The relative keys: ['C Major', 'A Minor']
The relative keys: ['D Major', 'B Minor']
The relative keys: ['E Major', 'C♯ Minor']
The relative keys: ['F Major', 'D Minor']
The relative keys: ['G Major', 'E Minor']
The relative keys: ['A Major', 'F♯ Minor']
The relative keys: ['B Major', 'G♯ Minor']
The relative keys: ['C♯ Major', 'A♯ Minor']
The relative keys: ['F♯ Major', 'D♯ Minor']
The relative keys: ['C Minor', 'Eb Major']
The relative keys: ['F Minor', 'Ab Major']
The relative keys: ['G Minor', 'Bb Major']
The relative keys: ['Cb Major', 'Ab Minor']
The relative keys: ['Db Major', 'Bb Minor']
The relative keys: ['Gb Major', 'Eb Minor']


In [ ]:
#set a threshold if any for rejecting the least occuring notes
# min_occurences = int(0.02*sum(note_occurences)) #2% removal
min_occurences = 0
note_soup = [note for note in note_desc if note[1]>min_occurences]

In [ ]:
#A dictionary of how many times each unique note appeared in all octaves
note_soup_dict = {}
midi_soup_dict = {}
for note, count in note_soup:
    # Extract the note name without octave
    note_name_without_octave = note[:-1]
    note_soup_dict[note_name_without_octave] = note_soup_dict.get(note_name_without_octave,0)+count
    midi_soup_dict[str(librosa.note_to_midi(note_name_without_octave))] = midi_soup_dict.get(str(librosa.note_to_midi(note_name_without_octave)),0)+count

In [ ]:
#Notes sorted from most to least according to the occurrences
#notes
sorted_note_data = {k: v for k, v in sorted(note_soup_dict.items(), key=lambda item: item[1], reverse=True)}
# midi numbers
sorted_midi_data = {k: v for k, v in sorted(midi_soup_dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
#function to arrange the notes in the order it appears in the song
def remove_duplicates_preserve_midi_order(input_list):
    seen = set()  # Create an empty set to store seen elements
    output_list = []  # Create an empty list to store unique elements while preserving order

    for item in input_list:
        if item not in seen:
            seen.add(item)  # Add the item to the set of seen elements
            output_list.append(str(librosa.note_to_midi(item)))  # Add the item to the output list

    return output_list

In [ ]:
uniq_significat_notes = [note[:-1] for note,occur in note_soup]
uniq_significat_midi_sets = remove_duplicates_preserve_midi_order(uniq_significat_notes)

In [ ]:
#create a dictionary of scores to observe the notes that come closest to the key
key_score_dict = {}
for key in key_scales:
  key_score_dict[key]= 0 # intialize with zero

In [ ]:
def change_value_list_to_midi(value_set):
  return [str(librosa.note_to_midi(x) )for x in value_set]

In [ ]:
#calculating the arbitrary key score
for key,value_set in key_scales.items():
  value_list= change_value_list_to_midi(value_set)
  uniq_notes_set = set(uniq_significat_midi_sets)
  dif_from_keys = set(value_list)-uniq_notes_set #stores values present in value_list not in uniq_notes_set
  dif_in_keys = uniq_notes_set-set(value_list) #stores the notes that are in uniq_notes_set not in value_list
  key_error_score = sum([sorted_midi_data.get(key_def,0) for key_def in list(dif_from_keys)]) #stores the number of notes that do not appear in the note_data, this is trivial all answers will be 0
  different_midi_error = sum([sorted_midi_data.get(note_def,0) for note_def in list(dif_in_keys)]) #stores the number of extra notes appearing in the note_data, not in key scale
  key_score_dict[key] = different_midi_error/len(dif_in_keys) if not len(dif_in_keys) == 0 else 0 #on average how often do the extra notes appear in the song that are not in the key scale

In [ ]:
#arranging the score list from least to most difference
sorted_key_score_dict = {k: v for k, v in sorted(key_score_dict.items(), key=lambda item: item[1])}

In [ ]:
#Handling for keys that receive the same error score

#Get the key with the closest match of notes
index =  min(sorted_key_score_dict,key=lambda k:sorted_key_score_dict[k])
#what is the error value of the closest key
min_data = sorted_key_score_dict.get(index)
#which other keys have the same error values
keys_with_value = [key for key, value in sorted_key_score_dict.items() if value == min_data]

In [ ]:
#storing in a list the keys that have the lowest error rate
probable_keys = [key_scales[key] for key in keys_with_value]
probable_midi_keys = [list(librosa.note_to_midi(note_list)) for note_list in probable_keys]

#converting the unique notes of the song to midi numbers to identify the lowest note
unique_midi = librosa.note_to_midi(unique_notes)

#sort from smallest to largest
unique_midi.sort()

#take the midi numbers back to note name
sorted_notes = librosa.midi_to_note(unique_midi)

#remove the octaves from the notes
# sort_note = [note[:-1] for note in sorted_notes]
sort_note = [librosa.note_to_midi(note[:-1]) for note in sorted_notes]

# Convert the sublists to sets for faster checking
# key_sets = [set(keys) for keys in probable_midi_keys]
key_sets = [keys for keys in probable_midi_keys]

# Find the first note value that appear in both sublists
# common_value = [value for value in sort_note if all(value in sublist for sublist in librosa.note_to_midi(key_sets))][0]
common_value = [value for value in sort_note if all(value in sublist for sublist in probable_midi_keys)][0]

#find the position of the common_value that appears first in both the sublist
positions = [(index, sublist.index(common_value)) for index, sublist in enumerate(probable_midi_keys) if common_value in sublist]

#extract the position of the sublist that have the common note appearing first in the list
min_index = min((index for index, position in positions), key=lambda x: positions[x][1])

#extract the values of the keys based on the index
key_val = probable_midi_keys[min_index]

# key_val = list(librosa.midi_to_note(key_val))
# key_val = [note for note in key_val]

#extract the the name of the keys based on the notes
detected_musical_key = [key for key, value in key_scales.items() if list(librosa.note_to_midi(value)) == key_val]
accepted_notes = key_scales.get(detected_musical_key[0])

#the sets of notes converted to midi notes and sorted
accepted_midi = [str(librosa.note_to_midi(n)) for n in accepted_notes]
accepted_midi.sort()

print(f"The probable keys were {keys_with_value},\nlowest note {librosa.midi_to_note(common_value)} which appears first in Detected key: {detected_musical_key[0]}\nwith notes: {accepted_notes}")

The probable keys were ['C Major', 'A Minor'],
lowest note C0 which appears first in Detected key: C Major
with notes: ['C', 'D', 'E', 'F', 'G', 'A', 'B']


In [ ]:
#finds the notes in the music that does not appear in the key and then transforms it to the closest note in the key

def find_closest_midi(input_number, num_list=accepted_midi):
    closest_numbers = []
    min_difference = float('inf')

    for num in num_list:
        num = int(num)
        difference = abs(input_number - num)

        if difference < min_difference:
            min_difference = difference
            closest_numbers = [num]
        elif difference == min_difference:
            closest_numbers.append(num)

    return closest_numbers

In [ ]:
#Quantize the note according to the closest frequency

not_key_note = list(set(uniq_significat_midi_sets).difference(accepted_midi))#notes in key not in the song
note_name = [str(librosa.note_to_midi(note[:-1])) for note in y_notes]
index_to_change = [np.where(np.array(note_name) == not_key_note[i])[0] for i in range(len(not_key_note))]
indices = np.concatenate(index_to_change)
indices_sort = np.sort(indices)
freq_to_change = [freq[index] for index in  indices_sort ]

In [ ]:
#creates a list of the notes that has been transformed using find_closest_midi function

def quantize_note(note:str,freq=0):

  pure_note = note[:-1]
  pure_scale = note[-1:]
  pure_midi = librosa.note_to_midi(pure_note)
  closest_midis = find_closest_midi(pure_midi)
  closest_midi_notes = librosa.midi_to_note(closest_midis)
  result_midis = [c_n[:-1]+pure_scale for c_n in closest_midi_notes]
  result_frequency = [librosa.note_to_hz(midi) for midi in result_midis]
  if freq:
    diff = abs(result_frequency - freq)
    closest=result_frequency[np.where(diff == min(diff))[0][0]]
    return  librosa.hz_to_midi(closest)

  return librosa.note_to_midi(result_midis[-1])

degrees = []
frequencies = []
for i,(note,fre) in enumerate(zip(y_notes,freq)):
  if i in indices_sort:
    closest_key_note= quantize_note(note,fre)
  else:
    closest_key_note = quantize_note(note)
  degrees.append(closest_key_note)




In [ ]:
detected_key = detected_musical_key[0]
key_note = detected_key.split(" ")[0]
key_note_in_soup = None
for (note,occurence) in note_soup:
  if librosa.note_to_midi(note[:-1]) == librosa.note_to_midi(key_note):
    key_note_in_soup=note
    break #gets the first note with octave corresponding to key_note

#the quantized notes are now reset to the correct octave
quantized_degrees = [int(round(deg,1)) for deg in degrees]

In [ ]:
#Time Quantization

bpm = tempo
beat_per_sec = bpm/60
second_per_beat = 60/bpm
grids_per_beat = 4
seconds_per_grid = (second_per_beat/4)
msec_per_grid = (second_per_beat/4)*1000

corrected_start_time_grid = [round(x/seconds_per_grid) for x in onset_times]
duplicates = [i for i, x in enumerate(corrected_start_time_grid) if corrected_start_time_grid.count(x) > 1]
index_to_change_new = [duplicates[i] for i in range(len(duplicates)) if i%2!=0]

#takes the index to change and then pushes the value back by one value: (the corrected_start_time_grid is being changed in this step)
for x in index_to_change_new:
  runnig_index = x
  while runnig_index>=0:

    if corrected_start_time_grid[runnig_index] == corrected_start_time_grid[runnig_index-1] and corrected_start_time_grid[runnig_index-1]!=0:
      corrected_start_time_grid[runnig_index-1]=corrected_start_time_grid[runnig_index-1]-1
      runnig_index = runnig_index-1

    else:
      break;

#find the positions(index) where the value of the grid is zero
something = [i for i,x in enumerate(corrected_start_time_grid) if x == 0]

#increasing the values of the index by an increment of 1 from the index of the last value till the index of the first value
for s in something[::-1]:
  if s==0:
    break;
  for i in range(s, len(corrected_start_time_grid)):
    corrected_start_time_grid[i] += 1

#change the grid values to seconds and then beat for midi conversion
corrected_start_time = [x*seconds_per_grid for x in corrected_start_time_grid]
corrected_start_time_beat = [x * beat_per_sec for x in corrected_start_time]
duration_in_second = [onset_times[i]-onset_times[i-1] for i in range(1,len(onset_times))]
corrected_start_time_beat.pop()

#truncate over-lapping notes
for i in range(len(corrected_start_time)-1):
  if corrected_start_time[i] + duration_in_second[i] > corrected_start_time[i+1]:
    duration_in_second[i] =  corrected_start_time[i+1] - corrected_start_time[i]

duration_beat_trunc = [x * beats_per_sec for x in duration_in_second]


In [ ]:
second_per_beat = 60/bpm #1 beat = 1 second
grids_per_beat = 4 # 1 beat = 4 grids = 1 second
seconds_per_grid = (second_per_beat/grids_per_beat) #1 grid = 0.25 seconds

grid_times = [round(x/seconds_per_grid) for x in onset_times] #the grid index of where the onset times will align #this is just a count
last_grid_val = max(grid_times)
length = last_grid_val + 4 if last_grid_val % 4!=0 else last_grid_val
metronome= range(0,length)
metronome_grid = [i for i, x in enumerate(metronome) if i % 4 == 0]
metronome_grid_seconds = [x*seconds_per_grid for x in metronome_grid]#converted the positions into seconds
metronome_grid_beat = [x * beat_per_sec for x in metronome_grid_seconds]#converted the seconds into beats

metronome_degrees = [most_common_note]*(len(metronome_grid_beat)-1)
metronome_duration = [metronome_grid_beat[1]/2]*(len(metronome_grid_beat)-1)
metronome_vol = [100] * (len(metronome_grid_beat)-1)


Chord generator

In [ ]:
def generate_chords(scale:str):
  key_notes=key_scales.get(scale)
  length = len(key_notes)
  triad_chords = []
  for i in range(length):
    triad_chords.append([key_notes[i%length],key_notes[(i+2)%length],key_notes[(i+4)%length]])
  return triad_chords

# Hold

In [ ]:
#Hold chord
def sort_by_note_position(triad,note):
    return triad.index(note)

key_chords = generate_chords(detected_key)
midi_chords=[]
for c_l in key_chords:
  cell=[]
  for n in c_l:
    cell.append(librosa.note_to_midi(n))
  midi_chords.append(cell)

q_notes = librosa.midi_to_note(degrees)

chord_prog =[]
chord_duration = []
chord_start = []
detected_chords = []
this_chord = []
prev_chord = []
start = 0

# chord_octave = max(int(librosa.midi_to_note(min(degrees))[-1:]),3) #the octave of the lowest note -> 2
chord_octave = int(librosa.midi_to_note(most_common_note)[-1])
for i,(note, time) in enumerate(zip(q_notes, corrected_start_time_beat)): #corrected_start_time_beat is the time quantized beat time
  if i == len(q_notes)-1:
    chord_start.append(corrected_start_time_beat[start]) #the 0th index from the start_time_beat which is 0.0
    chord_duration.append(corrected_start_time_beat[i]-corrected_start_time_beat[start])# the difference between the first chord starting point and the second chord starting point
    chord_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave)  for x in prev_chord])

  if not detected_chords:
    detected_chords = midi_chords # fills up the empty list with all the chords from the detected key of the song
  print(detected_chords)
  prev_chord = detected_chords[0] #the first triad of that chord because we know that the song starts with the root note which is also the first note in the 1st chord
  detected_chords = sorted([x for x in detected_chords if librosa.note_to_midi(note[:-1]) in x], key=lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1]))) #checks every note in the song with the notes in the chords and then returns the chord that has the note sorted in the first position of the song
  this_chord  = detected_chords[0] if detected_chords else None #returns the first chord of the set of chords as long as the notes in the song are also in the chord, if the notes of the song do not exist in the chord, the chord will return none
  print(note,librosa.note_to_midi(note[:-1]),prev_chord,this_chord)
  print(detected_chords)


  if not this_chord:#what happens if the note in the song does not coincide with the note in the chord and returns a none
    chord_start.append(corrected_start_time_beat[start]) #the 0th index from the start_time_beat which is 0.0
    chord_duration.append(corrected_start_time_beat[i]-corrected_start_time_beat[start])# the difference between the first chord starting point and the second chord starting point

    start = i
    # chord_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave) if x>12 else librosa.midi_to_note(x+12)[:-1]+str(chord_octave)  for x in prev_chord])
    chord_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave)  for x in prev_chord])
    detected_chords = midi_chords
    detected_chords = sorted([x for x in detected_chords if librosa.note_to_midi(note[:-1]) in x], key=lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1])))

In [ ]:
#sustaining chord
def sort_by_note_position(triad,note):
    return triad.index(note)

key_chords = generate_chords(detected_key)


midi_chords=[]
for c_l in key_chords:
  cell=[]
  for n in c_l:
    cell.append(librosa.note_to_midi(n))
  midi_chords.append(cell)

q_notes = librosa.midi_to_note(degrees)

chord_spike_prog =[]
chord_spike_duration = []
chord_spike_start = []
detected_chords = []
this_chord = []
prev_chord = []
start = 0

# chord_octave = max(int(librosa.midi_to_note(min(degrees))[-1:]),3) #the octave of the lowest note -> 2
# lowest_chord_octave = int(librosa.midi_to_note(min(degrees))[-1:])
# chord_octave = max(lowest_chord_octave, lowest_chord_octave+2)
chord_octave = int(librosa.midi_to_note(most_common_note)[-1])
for i,(note, time) in enumerate(zip(q_notes, corrected_start_time_beat)): #corrected_start_time_beat is the time quantized beat time
  if not detected_chords:
    detected_chords = midi_chords # fills up the empty list with all the chords from the detected key of the song
  # prev_chord = detected_chords[0] #the first triad of that chord because we know that the song starts with the root note which is also the first note in the 1st chord
  detected_chords = sorted([x for x in midi_chords if librosa.note_to_midi(note[:-1]) in x], key=lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1]))) #checks every note in the song with the notes in the chords and then returns the chord that has the note sorted in the first position of the song
  chord_spike  = detected_chords[0]  #returns the first chord of the set of chords as long as the notes in the song are also in the chord, if the notes of the song do not exist in the chord, the chord will return none
  chord_spike_start.append(corrected_start_time_beat[i])
  if i < len(corrected_start_time_beat)-1: #need to find another measure for duration
    chord_spike_duration.append(corrected_start_time_beat[i+1]-corrected_start_time_beat[i])
  else :
    chord_spike_duration.append(.25)
  chord_spike_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave) if x>12 else librosa.midi_to_note(x+12)[:-1]+str(chord_octave)  for x in chord_spike])



In [ ]:
prev_chord = None
smoothen_chords = []
smoothen_start =[]
smoothen_duration=[]
start_i=0
dur_i=0
for i,(chord, start,dur) in enumerate(zip(chord_spike_prog,chord_spike_start,chord_spike_duration)):
  if not prev_chord:

    prev_chord = chord
    start_i=start
    dur_i=0

  if prev_chord != chord:
    smoothen_chords.append(prev_chord)
    smoothen_start.append(start_i)
    smoothen_duration.append(dur_i)
    start_i=start
    dur_i=dur
    prev_chord = chord

  else:
    dur_i += dur


smoothen_vol = [100] * (len(smoothen_chords))



In [ ]:
#Writing the midi file

# Create a MIDIFile object
midi = MIDIFile(numTracks=6)

# Set track and channel for the audio
track = 0
channel = 1
program = 0 #
time = 0
track_name = "Piano"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for pitch, time, duration, volume in zip(quantized_degrees, corrected_start_time_beat, duration_beat_trunc, norm_vol):
  # print(time,duration)
  midi.addNote(track, channel, pitch, time, duration, volume)


# Set track and channel for the Chord(Major-third)(Hold)
track = 2
channel = 3
program = 0# piano
time = 0
track_name = "Major_third_chord_Hold"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(chord_prog, chord_start, chord_duration, norm_vol):
  # print(time,duration)
  for c_n in chord:
    midi.addNote(track, channel, librosa.note_to_midi(c_n), time, duration, volume)

# Set track and channel for the Hold chord bassline
track = 1
channel = 2
program = 32
time = 0
track_name = "Hold_Bass_line"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(chord_prog, chord_start, chord_duration, norm_vol):
  # print(time,duration)
    midi.addNote(track, channel, min(librosa.note_to_midi(chord)), time, duration, volume)



# Set track and channel for the Chord(Major-third)(Sustain)
track = 4
channel = 5
program = 0#piano
time = 0
track_name = "Major_third_chord_sustain"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo = int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(smoothen_chords, smoothen_start, smoothen_duration, smoothen_vol):
   for c_n in chord:
    midi.addNote(track, channel, librosa.note_to_midi(c_n), time, duration, volume)

# Set track and channel for the Smoothen chord bassline
track = 3
channel = 4
program = 32
time = 0
track_name = "Smoothen_Bass_line"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(smoothen_chords, smoothen_start, smoothen_duration, smoothen_vol):
  # print(time,duration)
    midi.addNote(track, channel, min(librosa.note_to_midi(chord)), time, duration, volume)


# Set track and channel for the metronome
track = 5
channel = 6
program = 115
time = 0
track_name = "Metronome"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
metronome_grid_beat.pop()
# Add the notes to the MIDI file
for pitch, time, duration, volume in zip(metronome_degrees, metronome_grid_beat, metronome_duration, metronome_vol):
  midi.addNote(track, channel, pitch, time, duration, volume)




# Write the MIDI file to disk
# output_file = "Note_time_quant_bass_metro " + audio_file.split('/')[2].split(".")[0]+ str(estimated_tempo)+'bpm'+ '.mid'
output_file = detected_musical_key[0] +'_'+ str(tempo)+'bpm'+'_'+audio_file.split('/')[2].split(".")[0]+ '.mid'
with open(output_file, 'wb') as file:
    midi.writeFile(file)

In [ ]:
metronome_degrees

[62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,


In [ ]:
q_notes

array(['B5', 'E4', 'E4', 'E5', 'E4', 'E4', 'E4', 'D4', 'E4', 'E4', 'F4',
       'F4', 'G5', 'D4', 'D4', 'D4', 'D4', 'C4', 'D4', 'D4', 'C4', 'A3',
       'D4', 'E4', 'E4', 'E4', 'E4', 'D4', 'E4', 'E4', 'E4', 'E4', 'E4',
       'E4', 'E4', 'F4', 'D4', 'D4', 'D4', 'C3', 'D4', 'D4', 'D4', 'C4',
       'D4', 'D4', 'C4', 'C4', 'D4', 'A3', 'B3', 'C4', 'C5', 'B4', 'B4',
       'A4', 'G4', 'B4', 'B4', 'B4', 'B4', 'E4', 'F4', 'F4', 'F4', 'F4',
       'F4', 'F4', 'F4', 'E4', 'E4', 'C4', 'A4', 'E4', 'G4', 'A4', 'E4',
       'D4', 'D4', 'D4', 'F5', 'G3', 'A3', 'B3', 'E4', 'E4', 'F3', 'A4',
       'G4', 'F4', 'G4', 'A4', 'E4', 'D4', 'D4', 'D4', 'A3', 'C4', 'D4',
       'E5', 'C4', 'B5', 'G4', 'G4', 'G4', 'C5', 'A4', 'G4', 'E4', 'A4',
       'G4', 'F4', 'E5', 'G4', 'A4', 'A4', 'A4', 'G4', 'G4', 'A4', 'E4',
       'C4', 'C4', 'D4', 'E4', 'D4', 'D4', 'C4', 'G5', 'C4', 'E4', 'E4',
       'E4', 'D4', 'E4', 'D4', 'E4', 'F4', 'F4', 'F4', 'F4', 'C4', 'D4',
       'D4', 'D4', 'C4', 'D4', 'D4', 'A3', 'D4', 'E

In [ ]:
chord_prog

[['E4', 'G4', 'B4'],
 ['D4', 'F4', 'A4'],
 ['E4', 'G4', 'B4'],
 ['F4', 'A4', 'C4'],
 ['G4', 'B4', 'D4'],
 ['C4', 'E4', 'G4'],
 ['D4', 'F4', 'A4'],
 ['A4', 'C4', 'E4'],
 ['D4', 'F4', 'A4'],
 ['E4', 'G4', 'B4'],
 ['D4', 'F4', 'A4'],
 ['E4', 'G4', 'B4'],
 ['D4', 'F4', 'A4'],
 ['C4', 'E4', 'G4'],
 ['D4', 'F4', 'A4'],
 ['C4', 'E4', 'G4'],
 ['D4', 'F4', 'A4'],
 ['C4', 'E4', 'G4'],
 ['D4', 'F4', 'A4'],
 ['B4', 'D4', 'F4'],
 ['C4', 'E4', 'G4'],
 ['B4', 'D4', 'F4'],
 ['A4', 'C4', 'E4'],
 ['E4', 'G4', 'B4'],
 ['F4', 'A4', 'C4'],
 ['A4', 'C4', 'E4'],
 ['G4', 'B4', 'D4'],
 ['A4', 'C4', 'E4'],
 ['D4', 'F4', 'A4'],
 ['G4', 'B4', 'D4'],
 ['A4', 'C4', 'E4'],
 ['E4', 'G4', 'B4'],
 ['F4', 'A4', 'C4'],
 ['G4', 'B4', 'D4'],
 ['F4', 'A4', 'C4'],
 ['G4', 'B4', 'D4'],
 ['A4', 'C4', 'E4'],
 ['D4', 'F4', 'A4'],
 ['C4', 'E4', 'G4'],
 ['D4', 'F4', 'A4'],
 ['C4', 'E4', 'G4'],
 ['G4', 'B4', 'D4'],
 ['A4', 'C4', 'E4'],
 ['E4', 'G4', 'B4'],
 ['A4', 'C4', 'E4'],
 ['G4', 'B4', 'D4'],
 ['F4', 'A4', 'C4'],
 ['E4', 'G4',